In [1]:
import tensorflow as tf
print(tf.__version__)


2.15.0


# Problem Statement

# Data Preprocessing

In [2]:
import tensorflow as tf
from tensorflow import keras

train = tf.keras.utils.image_dataset_from_directory('CNN Data/train', labels = 'inferred')
val = tf.keras.utils.image_dataset_from_directory('CNN Data/val', labels = 'inferred')

Found 9999 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.


# Deep Learning Model: CNN

In [3]:
types = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),

    tf.keras.layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),


    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    
    #tf.keras.layers.Dense(512, activation=tf.nn.relu),
    #tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)#, activation=tf.nn.softmax)
])

In [5]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.75):
      print("Cancelling training.")
      self.model.stop_training = True

callbacks = myCallback()

In [6]:
modelnet = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet',
)

model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomContrast(0.15),

    modelnet,

    tf.keras.layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),


    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    
    #tf.keras.layers.Dense(512, activation=tf.nn.relu),
    #tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)#, activation=tf.nn.softmax)
])

17225924/17225924 [==============================] - 2s 0us/step


In [7]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
model.fit(train, validation_data=val, epochs=60, callbacks=[callbacks])

Epoch 1/60


214/313 [===================>..........] - ETA: 1:18 - loss: 2.2593 - accuracy: 0.1431

# Evaluation of Model

In [ ]:
model.evaluate(val)

63/63 [==============================] - 2s 22ms/step - loss: 2.0659 - accuracy: 0.3910


[2.065906524658203, 0.39100000262260437]

# Hyperparameter Tuning